**Step 1:** Mount your Google Drive by clicking on "Mount Drive" in the Files section (panel to the left of this text.)

**Step 2:** Go to Runtime -> Change runtime type and select TPU.

**Step 3:** Create a folder in your Google Drive, and rename it to "vMalConv"

**Step 4:** Download the pre-processed training and test datasets.

In [ ]:
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_train.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_test.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/y_train.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/y_test.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/metadata.csv

--2021-10-13 20:20:20--  https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_train.dat
Resolving dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)... 52.219.100.72
Connecting to dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)|52.219.100.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8571600000 (8.0G) [application/x-www-form-urlencoded]
Saving to: ‘X_train.dat’

X_train.dat         100%[===================>]   7.98G  33.7MB/s    in 2m 47s  

2021-10-13 20:23:07 (49.0 MB/s) - ‘X_train.dat’ saved [8571600000/8571600000]

--2021-10-13 20:23:09--  https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_test.dat
Resolving dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)... 52.219.88.128
Connecting to dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)|52.219.88.128|:443... connected.
HTTP request sent, a

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

**Step 5:** Copy the downloaded files to vMalConv

In [ ]:
!cp /content/X_train.dat /content/drive/MyDrive/vMalConv/X_train.dat
!cp /content/X_test.dat /content/drive/MyDrive/vMalConv/X_test.dat
!cp /content/y_train.dat /content/drive/MyDrive/vMalConv/y_train.dat
!cp /content/y_test.dat /content/drive/MyDrive/vMalConv/y_test.dat
!cp /content/metadata.csv /content/drive/MyDrive/vMalConv/metadata.csv

**Step 6:** Download and install Ember:

In [ ]:
!wget https://github.com/endgameinc/ember/archive/master.zip
!unzip master.zip
!rm master.zip
!cp -r ember-master/* .
!rm -r ember-master
!pip install -r requirements.txt
!python setup.py install

--2021-10-13 20:30:28--  https://github.com/endgameinc/ember/archive/master.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/elastic/ember/archive/master.zip [following]
--2021-10-13 20:30:29--  https://github.com/elastic/ember/archive/master.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/elastic/ember/zip/master [following]
--2021-10-13 20:30:29--  https://codeload.github.com/elastic/ember/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11769696 (11M) [application/zip]
Saving to: ‘master.zip’

master.zip          100%[===================>]  11.22M  40.5MB/s  

**Step 7:** Read vectorized features from the data files.

In [ ]:
import ember
X_train, y_train, X_test, y_test = ember.read_vectorized_features("drive/MyDrive/vMalConv/")
metadata_dataframe = ember.read_metadata("drive/MyDrive/vMalConv/")

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


**Step 7:** Get rid of rows with no labels.

In [ ]:
labelrows = (y_train != -1)
X_train = X_train[labelrows]
y_train = y_train[labelrows]

In [ ]:
import h5py
h5f = h5py.File('X_train.h5', 'w')
h5f.create_dataset('X_train', data=X_train)
h5f.close()
h5f = h5py.File('y_train.h5', 'w')
h5f.create_dataset('y_train', data=y_train)
h5f.close()

In [ ]:
!cp /content/X_train.h5 /content/drive/MyDrive/vMalConv/X_train.h5
!cp /content/y_train.h5 /content/drive/MyDrive/vMalConv/y_train.h5

> **Exercise 1:** Complete the following code to create the architecture of MalConv in Keras:

In [23]:
def make_model():
  import tensorflow as tf
  from tensorflow import keras
  from tensorflow.keras import layers
  feature_size=2381
  tf.compat.v1.disable_eager_execution()

  keras.backend.clear_session()
  
  # Model architecture
  from tensorflow.keras import layers
  
 ### Your code -- Define the layers of MalConv ###
  model = tf.keras.Sequential()
  model.add(layers.InputLayer(input_shape=(1,feature_size)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(1500, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation='sigmoid'))

  model.compile(tf.keras.optimizers.Adam(learning_rate=0.001),
      loss='binary_crossentropy',
      metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])
  print(model.summary())
  return model
 

In [24]:
model = make_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 1, 2381)           0         
_________________________________________________________________
dense (Dense)                (None, 1, 1500)           3573000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1500)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              1501      
Total params: 3,574,501
Trainable params: 3,574,501
Non-trainable params: 0
_________________________________________________________________
None


**Step 8:** Partial fit the standardScaler to avoid overloading the memory:

In [25]:
from sklearn.preprocessing import StandardScaler
mms = StandardScaler()
for x in range(0,600000,100000):
  mms.partial_fit(X_train[x:x+100000])

In [26]:
X_train = mms.transform(X_train)

In [27]:
## Reshape to create 3 channels ##
import numpy as np
X_train = np.reshape(X_train,(-1,1,2381))
y_train = np.reshape(y_train,(-1,1,1))

> **Exercise 2:** Complete the following code to train the model for 30 epochs, with a batch size of 128, and 20% validation split. 

In [28]:
%tensorflow_version 2.x
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

save_dir = "drive/MyDrive/vMalConv/"

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

### Your code ###
model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
          loss='binary_crossentropy',
          metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])

history = model.fit(X_train, y_train,
                batch_size=128,
                epochs=30,
                  validation_split=.2,
                  callbacks=[callback]
                  )
# Save the weights #
model.save_weights (save_dir+'weights.h5')

# Save the model architecture #
model_json = model.to_json()
with open(save_dir+"model.json", "w") as json_file:
    json_file.write(model_json)

print("model saved.")

Train on 480000 samples, validate on 120000 samples
Epoch 1/30
479616/480000 [============================>.] - ETA: 0s - loss: 6.9983 - accuracy: 0.9407 - auc_1: 0.9465 - precision_1: 0.9395

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


480000/480000 [==============================] - 51s 107us/sample - loss: 6.9959 - accuracy: 0.9407 - auc_1: 0.9465 - precision_1: 0.9395 - val_loss: 3.5307 - val_accuracy: 0.9800 - val_auc_1: 0.9818 - val_precision_1: 0.9847
Epoch 2/30
480000/480000 [==============================] - 53s 111us/sample - loss: 7.9629 - accuracy: 0.9586 - auc_1: 0.9622 - precision_1: 0.9574 - val_loss: 2.7442 - val_accuracy: 0.9816 - val_auc_1: 0.9863 - val_precision_1: 0.9891
Epoch 3/30
480000/480000 [==============================] - 49s 101us/sample - loss: 8.8480 - accuracy: 0.9621 - auc_1: 0.9663 - precision_1: 0.9614 - val_loss: 20.6203 - val_accuracy: 0.9747 - val_auc_1: 0.9803 - val_precision_1: 0.9913
Epoch 4/30
480000/480000 [==============================] - 46s 95us/sample - loss: 13.7915 - accuracy: 0.9650 - auc_1: 0.9679 - precision_1: 0.9638 - val_loss: 6.9787 - val_accuracy: 0.9859 - val_auc_1: 0.9878 - val_precision_1: 0.9863
Epoch 5/30
480000/480000 [==============================] - 54

**Final Steps:** Download this Jupyter Notebook, and the saved model files.